In [127]:
from utils import load_embedding, parse_tup, cos_sim, cos_dist
from sklearn.decomposition import PCA
from tqdm.auto import tqdm
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd

In [142]:
TIME_FRAME = "biweekly"
subreddits, vectors  = load_embedding("/h/224/cameron/Political-Subreddit-Embedding/trained_embeddings/vecs_0.001_15.0.txt")
embedding = pd.concat([subreddits, vectors], axis=1).set_index("subreddit")
del subreddits, vectors
embedding

,1,2,3,4,5,6,7,8,9,10,...,141,142,143,144,145,146,147,148,149,150
subreddit,,,,,,,,,,,,,,,,,,,,,
SquaredCircle,0.023763,0.023023,-0.104653,-0.035605,-0.038755,0.049956,-0.125233,0.083531,0.049521,-0.040555,...,0.036282,0.132898,0.000326,0.012381,-0.026825,0.062032,0.097276,0.043152,0.045857,-0.006893
AskReddit,0.009698,0.015457,-0.032242,0.010191,0.021124,-0.019710,-0.048336,0.005754,0.168024,-0.056175,...,-0.071450,0.035780,0.047942,-0.179226,-0.024725,0.029362,0.033524,-0.065457,-0.007251,-0.150121
politics,-0.122210,-0.032153,-0.114660,-0.045690,-0.034402,-0.197407,-0.010298,0.088194,0.168841,-0.046625,...,-0.059785,0.072732,0.055958,0.042777,-0.096364,-0.028824,0.092465,-0.128514,0.017056,-0.106368
memes,-0.074882,-0.029766,-0.043359,0.042013,0.004274,0.044364,-0.021819,0.018184,0.102507,-0.036445,...,0.091629,0.006748,0.021836,-0.118849,0.028763,0.022777,-0.050563,-0.010202,0.034204,-0.098174
teenagers,0.081634,-0.019885,-0.113628,-0.042843,0.015160,0.052562,-0.005775,0.069234,0.046895,-0.072876,...,0.102843,-0.019443,0.096666,-0.123333,0.041919,0.039355,0.074338,-0.050082,0.050083,-0.083453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NRV,-0.065910,-0.069802,-0.020518,0.001313,-0.075752,-0.059001,0.006458,0.018057,0.191977,-0.107351,...,-0.088790,0.052766,0.033782,-0.036810,0.132314,-0.087799,-0.068353,-0.014926,-0.063411,-0.166090
UTT,-0.059624,-0.044377,-0.017682,-0.023010,-0.049664,-0.148291,0.007225,-0.058114,0.163091,-0.112971,...,-0.099901,0.045333,0.041086,-0.100427,0.024714,-0.029703,0.002404,0.001494,-0.074225,-0.179850
MMU,-0.051001,-0.108891,-0.014207,0.033639,-0.067141,-0.119037,-0.050742,-0.012071,0.197714,-0.139081,...,-0.029833,0.076010,-0.004877,-0.060819,0.045346,-0.000069,-0.059542,0.029543,0.011765,-0.129645


In [130]:
tqdm.pandas()
def find_closest(transformation,embedding,subreddit="politics",vec=None,exclude=[]):
    # Get subreddit
    assert subreddit or vec, "Must supply a subreddit name (that's in the data frame), or the vector representation of a subreddit"
    assert subreddit in embedding.index.values, "{} is not in the dataset".format(subreddit)
    sub_vec = embedding.loc[subreddit].to_numpy().flatten() if subreddit else vec
    transformed = sub_vec + transformation
    all_other_vecs = embedding[~embedding.index.isin(exclude+[subreddit])]
    similarity = np.apply_along_axis(lambda x : cos_sim(transformed,x), 1, all_other_vecs)
    winner_ind = similarity.argmax()
    
    return all_other_vecs.index.values[winner_ind], similarity[winner_ind]
    
def get_transformation(sub_a,sub_ap,embedding,add=False):
    """
        sub_a is to sub_ap
        sub_ap - sub_a
        returns transformation from a to ap
    """
    assert sub_a in embedding.index.values, "{} is not in the dataset".format(sub_a)
    assert sub_ap in embedding.index.values, "{} is not in the dataset".format(sub_ap)
    vec_a = embedding.loc[sub_a].to_numpy().flatten()
    vec_ap = embedding.loc[sub_ap].to_numpy().flatten()
    return vec_ap - vec_a if not add else vec_ap + vec_a

def get_possible_fits(a,ap,embedding,k=10):
    trans = get_transformation(a,ap,embedding)
    possible_fits = pd.DataFrame(map(lambda x : find_closest(trans,embedding,subreddit=x,exclude=[a,ap]),tqdm(embedding.index)))
    possible_fits = possible_fits.rename({0: "subreddit_to", 1: "trans_sim"},axis=1)
    possible_fits["subreddit_from"] = embedding.index
    possible_fits = possible_fits.sort_values('trans_sim',ascending=False)
    possible_fits = possible_fits[["subreddit_from","subreddit_to","trans_sim"]].reset_index(drop=True)
    possible_fits = possible_fits[~possible_fits["subreddit_from"].isin([a,ap])]
    return possible_fits.head(k)

a = "socialism"
ap = "Libertarian"
# p1 = get_possible_fits(a,ap,embedding,k=20)
# print(p1)
a = "SandersForPresident"
ap = "The_Donald"
p2 = get_possible_fits(a,ap,embedding,k=50)
print(p2)


           subreddit_from           subreddit_to  trans_sim
1                  asktrp             TheRedPill   0.801650
2                  askMRP         marriedredpill   0.786750
3              askanincel              Braincels   0.783108
4            Conservative            TheNewRight   0.782520
5                smuggies          Clownworldwar   0.781941
6               frenworld          Clownworldwar   0.778335
7              2ALiberals                NOWTTYG   0.778095
8          failed_Normies              Braincels   0.776397
9   SubforWhitePeopleOnly          Clownworldwar   0.775932
10   conservativecartoons            TheNewRight   0.774505
11      IncelsWithoutHate             askanincel   0.773299
12        Political_Tumor       drumpfisfinished   0.772836
13               walkaway            TheNewRight   0.771393
14       drumpfisfinished        Political_Tumor   0.768978
15       liberalgunowners                NOWTTYG   0.766519
16       ShitPoliticsSays        Politi

In [143]:
def analogy(sub_a,sub_ap,sub_b,embedding):
    """
        sub_a is to sub_ap, as sub_b is to ??
        sub_ap + sub_b - sub_a
        returns the subreddit that best answers the analogy
    """
    assert sub_a in embedding.index.values, "{} is not in the dataset".format(sub_a)
    assert sub_ap in embedding.index.values, "{} is not in the dataset".format(sub_ap)
    assert sub_b in embedding.index.values, "{} is not in the dataset".format(sub_b)
    vec_a = embedding.loc[sub_a].to_numpy().flatten()
    vec_ap = embedding.loc[sub_ap].to_numpy().flatten()
    vec_b = embedding.loc[sub_b].to_numpy().flatten()
    vec_bp = vec_ap + (vec_b - vec_a)
    all_other_vecs = embedding[~embedding.index.isin([sub_a,sub_ap,sub_b])]
    similarity = np.apply_along_axis(lambda x : cos_sim(x,vec_bp), 1, all_other_vecs)
    winner_ind = similarity.argmax()
    return all_other_vecs.index.values[winner_ind]

a = "socialism"
ap = "Libertarian"
b = "SandersForPresident"
print("r/{} is to r/{} as r/{} is to r/{}".format(a,ap,b,analogy(a,ap,b,embedding)))
ap = "ElizabethWarren"
b = "BaemyKlobaechar"
a = "SandersForPresident"
print("r/{} is to r/{} as r/{} is to r/{}".format(a,ap,b,analogy(a,ap,b,embedding)))

r/socialism is to r/Libertarian as r/SandersForPresident is to r/politics
r/SandersForPresident is to r/ElizabethWarren as r/BaemyKlobaechar is to r/centerleftpolitics


In [144]:
from itertools import combinations

def predict_analogies(analogy_names,embedding):
    fp = "/h/224/cameron/Political-Subreddit-Embedding/external_datasets/validation/{}.csv"
    data_frames = [pd.read_csv(fp.format(name)).sample(frac=0.5) for name in analogy_names]
    pairs = []
    for analogy_df in data_frames:
        tuples = [tuple(x) for x in analogy_df.to_numpy()]
        analogy_pairs = list(combinations(tuples,2))
        pairs += analogy_pairs
    pred = []
    actual = []
    tups = []
    for (a,ap),(b,bp) in tqdm(pairs):
        try: 
            p = analogy(a.strip(),ap.strip(),b.strip(),embedding)
            pred.append(p)
            actual.append(bp)
            tups.append(((a,ap),(b,bp)))
        except:
            pass
    return np.array(pred),np.array(actual), tups

def score(pred,actual):
    return (pred == actual).mean() * 100, (pred == actual).sum()

data_sets = ["football","basketball","baseball","hockey","populous_cities"]
# data_sets = ["populous_cities"]

pred, actual, tuples = predict_analogies(data_sets,embedding)
accuracy, num_correct = score(pred,actual)
print("Out of {} analogies, {} were predicted correctly ({}%)".format(len(pred),num_correct,accuracy))
for p,act,((a,ap),(b,_)) in zip(pred,actual,tuples):
    if not p==act: print("{} is to {} as {} is to {} (predicted as: {})".format(a,ap,b,act,p))



Out of 582 analogies, 418 were predicted correctly (71.8213058419244%)
NewOrleans is to Saints as LosAngeles is to LosAngelesRams (predicted as: 49ers)
NewOrleans is to Saints as tampa is to buccaneers (predicted as: miamidolphins)
NewOrleans is to Saints as LosAngeles is to Chargers (predicted as: 49ers)
NewOrleans is to Saints as oakland is to oaklandraiders (predicted as: 49ers)
LosAngeles is to LosAngelesRams as tampa is to buccaneers (predicted as: miamidolphins)
LosAngeles is to LosAngelesRams as LosAngeles is to Chargers (predicted as: oaklandraiders)
LosAngeles is to LosAngelesRams as oakland is to oaklandraiders (predicted as: 49ers)
indianapolis is to Colts as pittsburgh is to steelers (predicted as: buffalobills)
indianapolis is to Colts as tampa is to buccaneers (predicted as: miamidolphins)
indianapolis is to Colts as LosAngeles is to Chargers (predicted as: LosAngelesRams)
indianapolis is to Colts as oakland is to oaklandraiders (predicted as: 49ers)
nashville is to Tenn